# Explore Score Database Reader

This notebook shows how to read the database table of customer re-scores written by the Streams jobs, and allows exploring historical customer scores by graphing the scores over time for a customer, as it changes with new live events.

**This project contains Sample Materials, provided under license.  
Licensed Materials - Property of IBM.  
© Copyright IBM Corp. 2019. All Rights Reserved.  
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**


## Setup

If your DataSet has a different name, swap out LFE_SCORES below with the name of the appropriate DataSet.  Ensure that the TABLE name in the DataSet configuration is set appropriately.  The DataSet must match that set in the `streaming_analytics_lfe_pipeline` notebook.

In [ ]:
# Set this to the remote dataset name that is used for storing customer scores
CUSTOMER_SCORE_EVENTS_DATASET = "LFE_SCORES"


## Support Code for Dynamic and Interactive Widgets

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
import time
import threading
import dsx_core_utils, requests, jaydebeapi, os, io, sys
import pandas as pd
import getpass


# Update the graph based on a new customer selection, or just more recent data from the DB.
def updateGraph(cust, conn, table, g_out, g_label):
    plt.ioff()
    ax = plt.gca()

    if cust is not None and cust != "":
        df2 = pd.read_sql("select insertion_time,home_purchase_prob, relocate_prob from " + table + " where customer_id = ? order by insertion_time asc", params=(cust,),con=conn)
        if df2.shape[0] > 1:
            mbp = df2.apply(lambda r: (datetime.strptime(r['INSERTION_TIME'], '%Y-%m-%d %H:%M:%S.%f').timestamp() - datetime.now().timestamp())/60 , axis=1) 
            df2.set_index(mbp, inplace=True)
            df2.drop('INSERTION_TIME', axis=1, inplace=True)
            minx = min(min(mbp),-10)
    
            g_label.value = '<h3>Live Graph: Customer %s (%d scores)</h3>'%(cust,df2.shape[0])
    
            plt.plot(df2.index, df2['HOME_PURCHASE_PROB'],'.-')
            plt.plot(df2.index, df2['RELOCATE_PROB'],'.-')
            plt.hlines([0.4,0.5], minx, 0, colors='r')
            plt.xlim(minx, 0)
            plt.ylim(0,1)
            plt.title('Customer %s Score History' %(cust,))
            plt.xlabel('Minutes Before Present')
            plt.ylabel('Liklihood of Event')
            plt.legend(['HOME_PURCHASE_PROB', 'RELOCATE_PROB'])
            
            with g_out:
                plt.show(ax.figure)
        else:
            g_label.value = '<h3>Live Graph: Customer %s (%d scores)</h3>'%(cust,df2.shape[0])
            ax.clear()
            g_out.append_stdout("Not enough Scores.\n")
            
        g_out.clear_output(wait=True)
    else:
        g_label.value = '<h3>Live Graph</h3>'
        ax.clear()

# If the customer id has been changed, signal the thread to immediately go re-update the plot        
def changeCustomer(o, v, sw):
    if o != v:
        sw.set()

# If the button is pressed, signal the thread to terminate.
def stopThread(b, he):
    b.description = 'Stopping ...'
    b.disabled = True
    he.set()

# Update the list of "interesting" customers
def updateCustomers(conn, table, iclist):
    cdf = pd.read_sql("select customer_id, max(stddev(home_purchase_prob), stddev(relocate_prob)) as volatility from " + table + " group by customer_id order by volatility desc,customer_id asc", con=conn)
    iclist.append_stdout("Found %d customers\n"%(cdf.shape[0],))
    iclist.append_stdout("Top 15 most 'volatile':\n")
    iclist.append_stdout(cdf.head(15).to_string(index=False))
    iclist.clear_output(wait=True)

# Thread kernel: connect to the database and loop, updating the views, until asked to quit.
def threadfunc(g_out, g_label, dataset_name, button, he, sw, iclist, cidin):
    dataSet = dsx_core_utils.get_remote_data_set_info(dataset_name)
    dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
    if (sys.version_info >= (3, 0)):
      conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], {'user': dataSource['user'], 'password': dataSource['password'], 'clientProgramName': "dbg-rss-" + getpass.getuser()})
    else:
      conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
    
    sw.set()
    while not he.is_set():
        updateGraph(cidin.value, conn, dataSet['table'], g_out, g_label)
        updateCustomers(conn, dataSet['table'], iclist)
        sw.wait(5)
        sw.clear()

    conn.close()
    button.description = 'Stopped.'


## Widget Layout and Background Thread Startup

In [ ]:
# Lay out the required widgets
tt = widgets.Text(value="", placeholder='Customer Id', description='Customer:', layout={'width': '450px'}, disabled=False)
bb = widgets.Button(description='Stop Updating', button_style='danger', layout={'width': '450px'}, disabled=False)
gl = widgets.HTML(value='<h3>Live Graph</h3>')
go = widgets.Output(layout={'border':'1px solid black', 'width': '450px', 'height': '350px'})
ic = widgets.Output(layout={'border':'1px solid black', 'width': '450px', 'height': '350px'})
hbox = widgets.HBox([widgets.VBox([gl, go, tt]),\
                      widgets.VBox([widgets.HTML(value='<h3>Interesting Customers</h3>'), ic, bb])\
                    ])

# Signal used to ask the thread to terminate
halt_event = threading.Event()
# Signal used to more quickly get the plot to switch to a new customer
switch_customer = threading.Event()

# Register callbacks for the textbox and button
tt.observe(lambda c: changeCustomer(c['old'], c['new'], switch_customer), names='value', type='change')
bb.on_click(lambda b: stopThread(b, halt_event))

# Create and start the actual background thread that will update the graph
th = threading.Thread(target=threadfunc, args=(go, gl, CUSTOMER_SCORE_EVENTS_DATASET, bb, halt_event, switch_customer, ic, tt), name="Graph Updater")
th.start()    


## Explore Customer Score Evolution

Using the database of saved scores, we can look back in the history of each customer's scores, to see how they've changed over time. (Note that the timescale here is minutes before present of simulation time, since the events are being replayed from recorded events in the past.  As a result, events for a customer appear to happen much more frequently than they would in real life, in the minutes timescale, rather than the days or months timescale.)

On the left, enter a customer ID in the text field labelled 'Customer:'.  Assuming there is more than just one score, the graph of that customer's scores over time will appear, above the text field.  This graph will update periodically, as more events occur for that customer and cause them to get new scores.  The two red lines overlaid on the graph are the Significant Event thresholds currently set (40% and 50%).  Significant events are generated when a customer's score moves from above to the top line to below the bottom line, or vice versa.  (Also, in this demo, the first time a customer's score moves above the top line, even if it wasn't previously below the bottom line, a significant event will be generated.)

The 'significance' of a new customer score is determined in the `streaming_analytics_lfe_pipeline` notebook, in the `SignificantEvents` class definition.  Changing that cell to use a different definition of 'significance', and re-submitting the `significant_event_generation` Streams job from within that Notebook will change when these alerts are generated.  This could be a change to the thresholds, or a completely different way of determining 'signficance'.

To help find customer IDs that might prove interesting to look at, in this demo, on the right is a list of customers with high 'volatility' of scores over time.  That is, by some metric, they have scores that have changed a lot (or more than other customers).  The 'volatility' here is simply the standard deviation of the scores for that customer, over time.  Since we have two types of scores for each customer, the standard deviation is computed for each score type, and the displayed volatility is the maximum of those two values.  Obviously, some other, more meaningful metric could be chosen to find customers to explore, or the customer ID could be taken from the Significant Events Eventstreams feed, as alerts are generated live.  The current volatility calculation is done right in the SQL statement to retrieve the customer list, in the `updateCustomers()` function definition, above.

On the bottom right, there is a button that will stop the background thread from updating the graph or volatile customer list.  To re-start the updates, you'll need to re-set and re-run the Notebook.


In [ ]:
# Actually display the widgets
display(hbox)
